In [55]:
import pandas as pd
movies=pd.read_csv(r"C:\Users\hp\Downloads\movie_data\links.csv")
links=pd.read_csv(r"C:\Users\hp\Downloads\movie_data\movies.csv")
ratings=pd.read_csv(r"C:\Users\hp\Downloads\movie_data\ratings.csv")
tags=pd.read_csv(r"C:\Users\hp\Downloads\movie_data\tags.csv")

In [44]:
groupby_ratings = rating.groupby('movieId').agg({'rating': ['count', 'mean']}).reset_index()
groupby_ratings.columns = ['movieId', 'rating_count', 'rating_mean']
data = pd.merge(movie, groupby_ratings, on='movieId', how='inner')
filtered_data = data[data['rating_count'] > 50]
filtered_data.head()

,movieId,imdbId,tmdbId,rating_count,rating_mean
0,1,114709,862.0,215,3.920930
1,2,113497,8844.0,110,3.431818
2,3,113228,15602.0,52,3.259615
5,6,113277,949.0,102,3.946078
6,7,114319,11860.0,54,3.185185


In [54]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [57]:
import requests
import numpy as np
from bs4 import BeautifulSoup

def scrapper(imdbId):
    id = str(int(imdbId))
    n_zeroes = 7 - len(id)
    new_id = "0"*n_zeroes + id
    URL = f"https://www.imdb.com/title/tt{new_id}/"
    request_header = {'Content-Type': 'text/html; charset=UTF-8', 
                      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0', 
                      'Accept-Encoding': 'gzip, deflate, br'}
    response = requests.get(URL, headers=request_header)
    soup = BeautifulSoup(response.text, 'html.parser')
    imdb_rating = soup.find('span', attrs={'class' : "sc-bde20123-1 cMEQkK"})
    return imdb_rating.text if imdb_rating else np.nan

In [59]:
final_df= pd.merge(filtered_data,links, how='inner', on= 'movieId')
final_df

,movieId,imdbId,tmdbId,rating_count,rating_mean,imdb_ratings,title,genres
0,1,114709,862.0,215,3.920930,8.3,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,110,3.431818,7.1,Jumanji (1995),Adventure|Children|Fantasy
2,3,113228,15602.0,52,3.259615,6.6,Grumpier Old Men (1995),Comedy|Romance
3,6,113277,949.0,102,3.946078,7.2,Heat (1995),Action|Crime|Thriller
4,7,114319,11860.0,54,3.185185,6.8,Sabrina (1995),Comedy|Romance
...,...,...,...,...,...,...,...,...
431,106782,993846,106646.0,54,3.916667,NaN,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama
432,109374,2278388,120467.0,52,3.778846,NaN,"Grand Budapest Hotel, The (2014)",Comedy|Drama
433,109487,816692,157336.0,73,3.993151,NaN,Interstellar (2014),Sci-Fi|IMAX
434,112852,2015381,118340.0,59,4.050847,NaN,Guardians of the Galaxy (2014),Action|Adventure|Sci-Fi


In [60]:
r = []
for i in final_df['imdbId']:
    r.append(scrapper(i))
    

In [61]:
rat = pd.DataFrame({'rating':r})

In [62]:
final_df = pd.concat([final_df,rat], axis=1)

In [63]:
final_df['rating']= final_df['rating'].astype(float)

In [64]:
#Mention the movieId of the movie which has the highest IMDB rating.
final_df.loc[final_df['rating']==final_df['rating'].max()]

,movieId,imdbId,tmdbId,rating_count,rating_mean,imdb_ratings,title,genres,rating
53,318,111161,278.0,317,4.429022,7.1,"Shawshank Redemption, The (1994)",Crime|Drama,9.3


In [65]:
scifi_movies = final_df[final_df['genres'].str.contains('Sci-Fi', case=False)]

In [67]:
#Mention the movieId of the "Sci-Fi" movie which has the highest IMDB rating.

In [66]:
scifi_movies.loc[scifi_movies['rating']== scifi_movies['rating'].max()]

,movieId,imdbId,tmdbId,rating_count,rating_mean,imdb_ratings,title,genres,rating
424,79132,1375666,27205.0,143,4.066434,NaN,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,8.8
